In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from utils.initialise_weights import initialise_weights
from src.data_loader import *
from tqdm import tqdm
from utils.plot_to_tensorboard import plot_to_tensorboard
from utils.clear_folder import *
import torchshow as ts
import utils.plot_spectrograms as plot_spectrograms

# autoreload
%reload_ext autoreload
%autoreload 2

In [7]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels=64, out_channels=64,kernel_size=5, stride=1,padding=2,requires_sum=True,generator=True):
        super(ConvBlock, self).__init__()
        self.requires_sum = requires_sum
        self.block = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size,stride=stride, padding='same' if generator else padding),
            nn.PReLU() if generator else nn.LeakyReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size,stride=stride, padding='same' if generator else 2) if self.requires_sum else nn.Identity()) # changed in_channels to out_channels 15 Jul 2023 @ 15:39:28
        
    def forward(self, x):
        if self.requires_sum is True:
            out = self.block(x)
            return x + out
        else:
            out = self.block(x)
            return out
        


class Generator(nn.Module):
    def __init__(self,in_channel=2, out_channel=64, blocks=4):
        super(Generator, self).__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(in_channels=in_channel, out_channels=out_channel,kernel_size=11,stride=1,padding='same'),
            nn.PReLU()
        )

        self.blocks = nn.Sequential(*[ConvBlock() for _ in range(blocks)])
        self.conv = nn.Conv2d(in_channels=out_channel, out_channels=out_channel,kernel_size=5,stride=1,padding='same')

        self.terminal = nn.Sequential(
            ConvBlock(in_channels=out_channel, out_channels=out_channel,requires_sum=False),
            ConvBlock(in_channels=out_channel, out_channels=out_channel,requires_sum=False),
            nn.Conv2d(in_channels=out_channel, out_channels=3,kernel_size=11,stride=1,padding='same')
        )

    def forward(self, z):
        # https://de.wikipedia.org/wiki/Arctan2
        z = self.initial(z)
        out = self.blocks(z)
        out = self.conv(out)
        out = out + z
        out = self.terminal(out)

        mag  = out[:,:1,:,:]
        phase = torch.arctan2(out[:,1:2,:,:], out[:,2:,:,:])
        out = torch.cat([mag, phase],dim=1)
        return out


class Critic(nn.Module):
    def __init__(self,in_channels=2,out_channels=32,in_features=2048,out_features=512,in_features_final=2048,blocks=2):
        super(Critic, self).__init__()
        self.initial = nn.Sequential(
            ConvBlock(in_channels=2, out_channels=64,stride=1,padding=2, generator=False,requires_sum=False)
        )
        # [f(x) if condition else g(x) for x in sequence]
        self.blocks = nn.Sequential(*[ConvBlock(in_channels=64, out_channels=64,stride=2,generator=False,requires_sum=False)  \
                                    if blocks == 0 else ConvBlock(in_channels=64, out_channels=64,stride=2,generator=False,requires_sum=False) \
                                    for _ in range(blocks)])
        self.terminal = nn.Sequential(
            nn.Flatten(-1,1),
            nn.Linear(in_features=64 * 32 * 32 ,out_features=1024),
            nn.LeakyReLU(),
            nn.Linear(in_features=1024,out_features=1),
            nn.Sigmoid()
        )


    def forward(self,x):
        x = self.blocks(self.initial(x))

        print(x.shape)
        x = x.view(x.size(0), -1)

        print(x.shape)
        x = self.terminal(x)

        print(x.shape)


        return x.view(x.size(0), 1, 1, -1)


    @staticmethod
    def convert_to_complex(mag, angle):
        return mag*(np.cos(angle)+1j* np.sin(angle))

In [9]:
# Dimensionality Testing

B, C, H, W = 2, 1, 512, 512

x = torch.randn(B, C, H, W)
gen = Generator(1, 64, 4)
assert gen(x).shape == (B,2,H,W)
x = torch.randn(B, 2, H, W)
critic = Critic(blocks=4)
assert critic(x).shape == (B,1,1,1)
print("Dimensionality Testing Passed")

torch.Size([2, 64, 32, 32])
torch.Size([2, 65536])
torch.Size([2, 1])
Dimensionality Testing Passed


In [ ]:
# Real Data Testing
# Data Loader

dataset = 0

if dataset:

    path = '../data/AudioMNIST_Indicies/dummy_labels.csv'
    root_dir = '../data/AudioMNIST/'

else:
    path = '../data/AvianID_AcousticIndices/UK_AI.csv'
    root_dir='../data/UK_BIRD/'

# 

ds = AvianNatureSounds(annotation_file_path=path,
                       root_dir=root_dir,
                       key='habitat',
                       mode='stft',
                       length=5,
                       sampling_rate=44100,
                       n_fft=1024,
                       hop_length=512,
                       mel_spectrogram=None,
                       verbose=False,
                       fixed_limit=True)



train_loader = DataLoader(dataset=ds, batch_size=2, shuffle=True)



ground_truth, input_mag, _ = next(iter(train_loader))

# Visual Testing
enable = False
if enable:
    plot_spectrograms.plot_spectrograms(ground_truth,magnitude=False)


# Generator Check
initialise_weights(gen)
gen = Generator(1, 64, 4)
fake = gen(input_mag)
assert fake.shape == ground_truth.shape

fake.shape
# fake[:,:1,:,:] = phase
# fake[:,1:,:,:] = mag


In [ ]:
# Hyperparameter
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

lr = 3e-4
img_sz = 64
channel_img = 1
epochs = 3
feature_disc = 64
feature_gen = 64
noise_dim = 100

In [ ]:
gen = Generator(in_channel=1).to(device)
disc = Critic(blocks=4).to(device)

initialise_weights(gen)
initialise_weights(disc)

opt_gen = optim.Adam(gen.parameters(), lr=lr, betas=(0.5, 0.999))
opt_disc = optim.Adam(disc.parameters(), lr=lr, betas=(0.5, 0.999))

criterion = nn.MSELoss()

fixed_noise = torch.randn(32, noise_dim,1,1).to(device)

writer_fake = SummaryWriter(f'../data/runs/SPhaseGAN/fake')
writer_real = SummaryWriter(f'../data/runs/SPhaseGAN/real')
writer = SummaryWriter(f'../data/runs/SPhaseGAN/log')
step = 0

gen.train()
disc.train()

In [ ]:
def train():
    clear_folders('../data/runs/SPhaseGAN/fake/','../data/runs/SPhaseGAN/real/', '../data/runs/SPhaseGAN/log/')
    step = 0
    for epoch in range(epochs):
        loop = tqdm(train_loader, leave=False)
        for idx, (real_img, mag, _) in enumerate(loop):
            
            real_img = real_img.to(device)
            magnitude = mag.to(device)


            fake = gen(magnitude)

            ### Train Discriminator ###
            disc_real = disc(real_img)
            loss_disc_real = criterion(disc_real, torch.ones_like(disc_real))

            disc_fake = disc(fake)
            loss_disc_fake = criterion(disc_fake, torch.zeros_like(disc_fake))

            loss_disc = (loss_disc_real + loss_disc_fake) / 2
            disc.zero_grad()
            loss_disc.backward(retain_graph=True)
            opt_disc.step()

            ### Train Generator ###
            output = disc(fake)
            loss_gen = criterion(output, torch.ones_like(output)) / 2
            gen.zero_grad()
            loss_gen.backward()
            opt_gen.step()




            if idx == 0:
                print(
                    f'Epoch [{epoch / epochs}] \  '
                    f'Loss D: {loss_disc.item():.4f}, Loss G: {loss_gen.item():.4f}'
                )

    

            if idx % 50 == 0:
                plot_to_tensorboard(
                    writer = writer,
                    loss_critic = loss_disc.item(),
                    loss_gen = loss_gen.item(),
                    real = real_img[:,:1,:,:] ,
                    fake = fake[:,:1,:,:],
                    tb_step = step,
                    images = True
                )

                step += 1
                

                # with torch.no_grad():
                #     fake = gen(fixed_noise).reshape(-1, 1,64,64)
                #     data = real_img.reshape(-1, 1, 64, 64)

                #     img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
                #     img_grid_real = torchvision.utils.make_grid(data, normalize=True)

                #     writer_fake.add_image(
                #         'MNIST Fake Images', img_grid_fake, global_step=step
                #     )

                #     writer_real.add_image(
                #         'MNIST Real Images', img_grid_real, global_step=step
                #     )

                #     step += 1

$$
U(G)=\frac{1}{2} \mathbb{E}_{\mathbf{a} \sim p_{\mathbf{a}}(\mathbf{a}), \phi \sim p_\phi(\phi)}\left[(D(G(\mathbf{a}, \boldsymbol{\phi}), \mathbf{a})-1)^2\right]
$$

In [ ]:
train()